In [1]:
### no brand names and search terms vector

In [2]:
%load_ext Cython

In [3]:
%%time
%%cython
from gensim.models import doc2vec
import nltk
import re
def split_sentence (sentence):
    return re.split('\W+',sentence)
    
class MyDocs(object):
    def __iter__(self):
        for i, text in enumerate(open("txt_all_ns.txt")): # doesn't use brand name
            yield doc2vec.LabeledSentence(words=split_sentence(text), tags=['%s' % i])
            # Train the doc2vec model
cdef desp = MyDocs()
model = doc2vec.Doc2Vec(desp, size = 200, window = 8, min_count = 5, workers = 4)
model.save('distance_ns.model')

IOError: [Errno 2] No such file or directory: 'txt_all_nst.txt'

In [29]:
model = doc2vec.Doc2Vec.load('distance_ns.model')

In [4]:
import numpy as np
st_arrays = np.zeros((240760, 200))
for i,text in enumerate(open("txt_st.txt")): # input search terms 
    st_arrays[i]=model.infer_vector(split_sentence(text))   
st_arrays

NameError: name 'model' is not defined

In [40]:
st_arrays[1]

array([ 0.01927011, -0.05003396,  0.02516052,  0.03203654, -0.01559536,
       -0.06522521, -0.04188403,  0.02567124,  0.03131287, -0.03769345,
       -0.0223483 ,  0.04697653, -0.02646255, -0.0121666 ,  0.00923798,
        0.0553622 ,  0.00455855, -0.01982786, -0.03015708, -0.01524935,
       -0.06855255, -0.07331096, -0.00370974,  0.01564562,  0.03387643,
       -0.01482635,  0.02049366, -0.00907977,  0.00133557,  0.01640423,
       -0.0025919 ,  0.04307093, -0.01029093,  0.00932395, -0.06592943,
        0.05549867,  0.01965863, -0.02877121,  0.05057914, -0.021547  ,
       -0.02115977, -0.09191324,  0.00930923,  0.06013506,  0.0087908 ,
       -0.02479437,  0.03020176, -0.03683701,  0.05338268,  0.02232859,
        0.0085425 ,  0.02255772, -0.04790759, -0.02101214, -0.11276942,
        0.01213323,  0.00794208, -0.00828621, -0.06477316,  0.01860436,
        0.0182302 , -0.04577722,  0.06496342, -0.02041091, -0.00787426,
        0.04717857, -0.00504706, -0.02769087, -0.03646737, -0.02

In [41]:
## calculate the cosin distance between 2 vectors to generate the 3rd features
# Compute the cosine similarity values between the input text and all archived reviews
# cossims_with_input = map(lambda v: cossim(input_vec, v), model.docvecs) 
# need to chaneg the code into rows: calculate the cosin distance between the vectors in same rows
import numpy as np
# Calculate the cosine similarity between two vecotrs 
def cossim(v1, v2):
    return np.dot(v1, v2) / np.sqrt(np.dot(v1, v1)) / np.sqrt(np.dot(v2, v2))
cossims_with_st=np.zeros((240760,1))
for i in range(240760):
    cossims_with_st[i] = cossim(st_arrays[i], model.docvecs[i])

In [46]:
np.savetxt("distance_ns.csv",cossims_with_st, delimiter=",")

In [ ]:
cws_trn=coswithst.iloc[:num_train]
cws_tst=coswithst.iloc[num_train:]